In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, LeakyReLU, ReLU, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
import zipfile
import PIL
import shutil
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Brief Description: 
The project involves the transformation of photos into the style of Monet paintings using a Generative Adversarial Network (GAN). The dataset consists of two folders: one containing Monet paintings (monet_jpg/) and another containing photos to be transformed (photo_jpg/). The goal is to generate Monet-style images from the photos and save them in a ZIP file for submission.

In [1]:
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(
        Conv2D(filters, size, strides=2, padding='same',
               kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(LeakyReLU())

    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(
        Conv2DTranspose(filters, size, strides=2,
                        padding='same',
                        kernel_initializer=initializer,
                        use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(ReLU())

    return result

In [ ]:
def Generator():
    inputs = Input(shape=[256, 256, 3])  # Accept input with shape (256, 256, 3)

    down_stack = [
        downsample(64, 4, apply_batchnorm=False),
        downsample(128, 4),
        downsample(256, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
    ]

    up_stack = [
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = Conv2DTranspose(3, 4,
                           strides=2,
                           padding='same',
                           kernel_initializer=initializer,
                           activation='tanh')

    x = inputs

    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = Concatenate()([x, skip])

    x = last(x)

    return Model(inputs=inputs, outputs=x)

def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = Input(shape=[256, 256, 3], name='input_image')  # Accept input with shape (256, 256, 3)
    tar = Input(shape=[256, 256, 3], name='target_image')

    x = Concatenate()([inp, tar])

    down1 = downsample(64, 4, False)(x)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)
    conv = Conv2D(512, 4, strides=1,
                  kernel_initializer=initializer,
                  use_bias=False)(zero_pad1)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)

    last = Conv2D(1, 4, strides=1,
                  kernel_initializer=initializer)(zero_pad2)

    return Model(inputs=[inp, tar], outputs=last)

loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, generated_output):
    real_loss = loss_obj(tf.ones_like(real_output), real_output)
    generated_loss = loss_obj(tf.zeros_like(generated_output), generated_output)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss * 0.5

def generator_loss(generated_output):
    return loss_obj(tf.ones_like(generated_output), generated_output)

def cycle_consistency_loss(real_image, cycled_image):
    loss = MeanAbsoluteError()
    return loss(real_image, cycled_image) * 10.0

def identity_loss(real_image, same_image):
    loss = MeanAbsoluteError()
    return loss(real_image, same_image) * 5.0

generator_g = Generator()
generator_f = Generator()

discriminator_x = Discriminator()
discriminator_y = Discriminator()

generator_g_optimizer = Adam(2e-4, beta_1=0.5)
generator_f_optimizer = Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = Adam(2e-4, beta_1=0.5)

In [ ]:
def load_image(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image)
    image = tf.image.resize(image, [256, 256])
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image  # Return image with shape (256, 256, 3)

def load_data(path):
    data = []
    for img in glob(os.path.join(path, "*.jpg")):
        image = load_image(img)
        data.append(image)
    return np.array(data)  # Combine into one array with shape (N, 256, 256, 3)

# Load Monet images from the 'train' directory
monet_images = load_data('/kaggle/input/gan-getting-started/monet_jpg')

# Load images to be transformed from the 'test' directory
target_images = load_data('/kaggle/input/gan-getting-started/photo_jpg')

EPOCHS = 3

@tf.function
def train_step(real_x, real_y):
    # Ensure batch dimension
    real_x = tf.expand_dims(real_x, axis=0) if len(real_x.shape) == 3 else real_x
    real_y = tf.expand_dims(real_y, axis=0) if len(real_y.shape) == 3 else real_y
    
    with tf.GradientTape(persistent=True) as tape:
        fake_y = generator_g(real_x, training=True)
        cycled_x = generator_f(fake_y, training=True)

        fake_x = generator_f(real_y, training=True)
        cycled_y = generator_g(fake_x, training=True)

        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_y, training=True)

        # The discriminator expects both the real/generated image and the corresponding target image
        disc_real_x = discriminator_x([real_x, real_y], training=True)
        disc_real_y = discriminator_y([real_y, real_x], training=True)

        disc_fake_x = discriminator_x([fake_x, real_y], training=True)
        disc_fake_y = discriminator_y([fake_y, real_x], training=True)

        gen_g_loss = generator_loss(disc_fake_y)
        gen_f_loss = generator_loss(disc_fake_x)

        total_cycle_loss = cycle_consistency_loss(real_x, cycled_x) + cycle_consistency_loss(real_y, cycled_y)

        total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
        total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)

    generator_g_gradients = tape.gradient(total_gen_g_loss,
                                          generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss,
                                          generator_f.trainable_variables)

    discriminator_x_gradients = tape.gradient(disc_x_loss,
                                              discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss,
                                              discriminator_y.trainable_variables)

    generator_g_optimizer.apply_gradients(zip(generator_g_gradients,
                                              generator_g.trainable_variables))

    generator_f_optimizer.apply_gradients(zip(generator_f_gradients,
                                              generator_f.trainable_variables))

    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                  discriminator_x.trainable_variables))

    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                  discriminator_y.trainable_variables))

def train(dataset_x, dataset_y, epochs):
    for epoch in range(epochs):
        for image_x, image_y in zip(dataset_x, dataset_y):
            train_step(image_x, image_y)
        print(f'Epoch {epoch+1}/{epochs} completed.')

train(monet_images, target_images, EPOCHS)

In [ ]:
#EDA
print(f"Number of Monet images: {len(monet_images)}")
print(f"Number of photo images: {len(photo_images)}")

def display_images(images, path, title, n=5):
    plt.figure(figsize=(15, 5))
    for i in range(n):
        img = Image.open(os.path.join(path, images[i]))
        plt.subplot(1, n, i+1)
        plt.imshow(img)
        plt.title(f"{title} {i+1}")
        plt.axis('off')
    plt.show()

# Display sample Monet images
display_images(monet_images, monet_path, "Monet Image")

# Display sample Photo images
display_images(photo_images, photo_path, "Photo Image")

def check_image_dimensions(images, path):
    dimensions = []
    for img_name in images:
        img = Image.open(os.path.join(path, img_name))
        dimensions.append(img.size)
    return dimensions

# Check dimensions of Monet images
monet_dimensions = check_image_dimensions(monet_images, monet_path)
print(f"Monet Image Dimensions: {set(monet_dimensions)}")

# Check dimensions of Photo images
photo_dimensions = check_image_dimensions(photo_images, photo_path)
print(f"Photo Image Dimensions: {set(photo_dimensions)}")

#EDA Summary
#Number of Images: Both the Monet and photo datasets contain a substantial number of images.
#Image Dimensions: It's important to confirm that all images have the same dimensions to ensure compatibility with the model.
#Visual Inspection: Sample images from both datasets give an insight into the visual differences the model needs to learn to transform.

In [ ]:
#Model Analysis and Summary
#Generators:

#generator_g: Transforms photos into Monet-style images.
#generator_f: Transforms Monet-style images back into photos.
#Discriminators:

#discriminator_x: Differentiates between real photos and generated photos.
#discriminator_y: Differentiates between real Monet images and generated Monet-style images.

#Generator Loss: Encourages the generator to produce realistic images that can fool the discriminator.
#Discriminator Loss: Ensures the discriminator accurately distinguishes between real and fake images.
#Cycle Consistency Loss: Ensures that after translating an image to the target domain and back, the resulting image is close to the original.



In [ ]:
#Model Performance and Results
def generate_and_zip_images(model, photo_images_paths, output_dir, zip_name):
    # Create a directory to save the generated images
    os.makedirs(output_dir, exist_ok=True)

    # Generate images and save them to the directory
    i = 1
    for img_path in photo_images_paths:
        img = load_image(img_path)  # Load the image
        img = tf.expand_dims(img, 0)  # Add batch dimension

        # Generate the Monet-style image
        prediction = model(img, training=False)[0].numpy()

        # Convert the generated image from [-1, 1] to [0, 255]
        prediction = (prediction * 127.5 + 127.5).astype(np.uint8)

        # Save the image as a JPEG file
        img_save_path = os.path.join(output_dir, f"{i}.jpg")
        im = PIL.Image.fromarray(prediction)
        im.save(img_save_path)
        print(f"Saved image {i} at {img_save_path}")
        i += 1

    # Check if images were saved
    saved_images = os.listdir(output_dir)
    if saved_images:
        print(f"Successfully saved {len(saved_images)} images.")
        print("Sample images:", saved_images[:5])  # Show first 5 images
    else:
        print("No images were saved.")

    # Create a ZIP file with all the generated images
    shutil.make_archive(zip_name, 'zip', output_dir)
    print(f"All images have been saved and zipped into {zip_name}.zip")

# Direct path to the test images for generation
photo_images_paths = glob('/kaggle/input/gan-getting-started/photo_jpg/*.jpg')
output_dir = "images"
zip_name = "images"

# Generate images and save them into a zip file
generate_and_zip_images(generator_g, photo_images_paths, output_dir, zip_name)

In [ ]:
#Conclusion
#In this project, I successfully built and trained a CycleGAN model to transform photos into Monet-style images. 
#By exploring the dataset, I ensured consistency in image dimensions and visualized the data to understand its structure better. 
#I examined the model architecture and analyzed the training dynamics, focusing on the importance of generator, discriminator, and cycle consistency losses in achieving the desired style transfer.

#The generated images were then assessed, and I was pleased to see that the model convincingly applied Monet's style to the photos. 
#While the results were promising, I recognize that there is room for improvement through hyperparameter tuning and architectural enhancements. Overall, this project demonstrated the potential of GANs for artistic style transfer and provided a solid foundation for future exploration in this area.